# Homework Assignment 1: Write a Starting Prompt

This notebook shows you how to run the first homework example using Galileo.

## Configuration

To be able to run this notebook, you need to have a Galileo account set up, along with an LLM integration so that your prompt can be run against a model of your choice.

1. If you don't have a Galileo account, head to [app.galileo.ai/sign-up](https://app.galileo.ai/sign-up) and sign up for a free account
1. Once you have signed up, you will need to configure an LLM integration. Head to the [integrations page](https://app.galileo.ai/settings/integrations) and configure your integration of choice. The notebook assumes you are using OpenAI, but has details on what to change if you are using a different LLM.
1. Create a Galileo API key from the [API keys page](https://app.galileo.ai/settings/api-keys)
1. In this folder is an example `.env` file called `.env.example`. Copy this file to `.env`, and set the value of `GALILEO_API_KEY` to the API key you just created.
1. If you are using a custom Galileo deployment inside your organization, then set the `GALILEO_CONSOLE_URL` environment variable to your console URL. If you are using [app.galileo.ai](https://app.galileo.ai), such as with the free tier, then you can leave this commented out.


In [ ]:
# Install the galileo and python-dotenv package into the current Jupyter kernel
%pip install galileo python-dotenv

## Environment setup

To use Galileo, we need to load the API key from the .env file

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check that the GALILEO_API_KEY environment variable is set
if not os.getenv("GALILEO_API_KEY"):
    raise ValueError("GALILEO_API_KEY environment variable is not set. Please set it in your .env file.")

Next we need to ensure there is a Galileo project set up.

In [ ]:
from galileo.projects import create_project, get_project

PROJECT_NAME = "AI Evals Course - Homework 1"
project = get_project(name=PROJECT_NAME)
if project is None:
    project = create_project(name=PROJECT_NAME)

print(f"Using project: {project.name} (ID: {project.id})")

In this notebook, you will be using the LLM integration you set up in Galileo to run experiments, and generate synthetic data. The default model used is GPT-5.1, and this assumes you have configured an OpenAI integration.

If you have another integration set up, or want to use a different model, update this value.

In [ ]:
MODEL="gpt-5.1"

Finally lets create some unique names for the prompt and dataset, so these can be easily re-run multiple times.

In [ ]:
from datetime import datetime

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

PROMPT_NAME = f"Homework 1 Prompt - {current_time}"
DATASET_NAME = f"Homework 1 Dataset - {current_time}"

## Part1: Write an Effective System Prompt

### Create the prompt, and save it to Galileo

You can create [prompts](https://v2docs.galileo.ai/sdk-api/experiments/prompts) in Galileo to run against a dataset of inputs. These prompts use [mustache templates](https://mustache.github.io/), and when run against a dataset, the prompt is run against each row, replacing the templates parts of the prompt with values from the dataset.

Prompts can be created either as global to an organization, so they can be used by any member, or tied to a specific project. In this case, we will create a global prompt so it can be used in different projects for each homework assignment.

Once the prompt is created, a link will be output. You can follow the link to view the prompt in the Galileo console.

In [ ]:
from galileo import Message, MessageRole
from galileo.prompts import create_prompt, delete_prompt, get_prompt

# Define a system prompt. It is this prompt you need to configure
system_prompt = """
You are an expert chef recommending delicious and useful recipes. Present only one recipe at a time. If the user doesn't specify what ingredients they have available, assume only basic ingredients are available.Be descriptive in the steps of the recipe, so it is easy to follow.Have variety in your recipes, don't just recommend the same thing over and over.You MUST suggest a complete recipe; don't ask follow-up questions.Mention the serving size in the recipe. If not specified, assume 2 people.
"""

prompt = None

# Define a function to create the system prompt in Galileo, then call this.
# By using a function, we can easily re-run this code to update the prompt.
def set_up_prompt():
    """
    Create a prompt in Galileo using the system prompt defined above
    """
    global prompt

    # Start by getting the prompt if it already exists.
    # If it does, we can delete it and re-create, if not we create it.
    prompt = get_prompt(name=PROMPT_NAME)

    if prompt is not None:
        print(f"Prompt already exists with ID: {prompt.id}, deleting it to re-create.")
        prompt = delete_prompt(name=PROMPT_NAME)

    prompt = create_prompt(
        name=PROMPT_NAME,
        template=[
            Message(
                role=MessageRole.system,
                content=system_prompt,
            ),
            Message(role=MessageRole.user, content="{{input}}"),
        ],
    )

    # Output a link to view the prompt in Galileo
    print(f"Prompt created. You can view it at {os.environ.get('GALILEO_CONSOLE_URL', 'https://app.galileo.ai/').removesuffix('/')}/prompts/{prompt.id}")

set_up_prompt()

### Load the dataset and test out the prompt

There is a default dataset provided in the original homework. We can use this to test out the prompt.

Prompts are run against [datasets](https://v2docs.galileo.ai/sdk-api/experiments/datasets) created and maintained in Galileo. This code creates a Galileo dataset.

Like with prompts, datasets can be created at an organization or project level. In this case we will create one global to the organization.

Once the dataset is created, a link will be output. You can follow the link to view the dataset in the Galileo console.

In [ ]:
import csv
from urllib.request import urlopen

# Get the CSV file from the original GitHub repository for the course
source_path = "https://raw.githubusercontent.com/ai-evals-course/recipe-chatbot/refs/heads/main/data/sample_queries.csv"

# Load this csv file into a list of JSON objects with an `input` key
with urlopen(source_path) as resp:
    lines = (ln.decode("utf-8") for ln in resp)
    reader = csv.DictReader(lines)
    queries = []
    for row in reader:
        value = row.get("query")
        if not value:
            raise ValueError("CSV file must have a 'query' column.")
        queries.append({"input": value})

print(f"Loaded {len(queries)} queries from {source_path}")


In [ ]:
from galileo.datasets import get_dataset, create_dataset, delete_dataset

# Now we have the CSV file loaded, lets create a dataset. If the dataset already exists, we will delete it and re-create it.
dataset = get_dataset(
    name=DATASET_NAME
)

if dataset is not None:
    print(f"Dataset already exists with ID: {dataset.id}, deleting it to re-create.")
    dataset = delete_dataset(
        name=DATASET_NAME
    )

dataset = create_dataset(
    name=DATASET_NAME,
    content=queries,
)

print(f"Dataset created. You can view it at {os.environ.get('GALILEO_CONSOLE_URL', 'https://app.galileo.ai/').removesuffix('/')}/datasets/{dataset.id}")

### Run the prompt against the dataset using an experiment

Next we can run the prompt using the dataset in an [experiment](https://v2docs.galileo.ai/sdk-api/experiments/experiments). This will use the LLM integration you have set up earlier, and run each dataset row against the LLM using the prompt provided, saving the output to the experiment.

Experiments need a unique name, but if a name is in use then the current date and time is added. This means you can re-run this code and always get a new experiment.

This experiment run uses the model defined in the `PromptRunSettings`, using the model name you set earlier.

Experiments take time to run. The call to `run_experiment` will return as soon as the experiment has started. The link that is output will show the experiment, and you can monitor its progress from there.

In [ ]:
from galileo.experiments import run_experiment
from galileo.resources.models import PromptRunSettings

# Define a function to run the prompt experiment
# By using a function, we can easily re-run this code to start the experiment.
def run_experiment_using_prompt_and_dataset():
    # Create the experiment prompt run settings to define the model
    # Update the model_alias to the model you want to use for the experiment
    prompt_run_settings = PromptRunSettings(
        model_alias=MODEL
    )

    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    experiment_name = f"Homework 1 Experiment - {current_time}"

    # Run the experiment using the prompt and dataset we created
    results = run_experiment(
        experiment_name,
        dataset=dataset,
        prompt_template=prompt,
        project=PROJECT_NAME,
        prompt_settings=prompt_run_settings,
    )

    print(f"Experiment has started. You can view the experiment at {results['link']}")

run_experiment_using_prompt_and_dataset()

### View the results

Once the experiment has completed, you can view the generated output. Each row in the dataset is logged as a separate trace in the experiment.

<div>
<img src="./images/experiment-3-traces.webp" width="800"/>
</div>

Select each row to see the generated response from the prompt and dataset row.

<div>
<img src="./images/experiment-first-trace.webp" width="800"/>
</div>

In addition to the output from the LLM, you will also be able to see the number of tokens used, the time taken, and an estimated cost based off the published token prices for the LLM.

### Improve the system prompt

Now that you have a reliable way to run a dataset against your prompt, you can iterate on this to get the system prompt you want. Follow the instructions and examples in the [homework assignment](https://github.com/ai-evals-course/recipe-chatbot/tree/main/homeworks/hw1#part1-write-an-effective-system-prompt) to improve your prompt, and update the `system_prompt` below to reflect your desired changes.

Once updated, run the code to regenerate the prompt and run the experiment. Again, a link will be output so you can see the results once the experiment has completed.

In [ ]:
# Define the new system prompt
system_prompt = """
You are an expert chef recommending delicious and useful recipes. Present only one recipe at a time. If the user doesn't specify what ingredients they have available, assume only basic ingredients are available.Be descriptive in the steps of the recipe, so it is easy to follow. Have variety in your recipes, don't just recommend the same thing over and over.You MUST suggest a complete recipe; don't ask follow-up questions.

Create this recipe in the style of the swedish chef from the Muppets.
"""

# Update the prompt in Galileo
set_up_prompt()

# Run the prompt experiment again with the updated prompt
run_experiment_using_prompt_and_dataset()

## Part 2: Expand and Diversify the Query Dataset

### Expand the dataset manually

You can manually add rows to an existing dataset by passing them in as an array. Update the code below to include more rows, then run it to add these rows to the dataset.

Once added, a link to the dataset will be output so you can see the new rows.

In [ ]:
# Add new rows to the dataset
dataset = dataset.add_rows([
    {
        "input": "I love ramen. Can you suggest a recipe for it using pork belly?",
    },
    {
        "input": "What are some good gluten and dairy free birthday cake ideas?",
    },
])

print(f"Dataset extended. You can view it at {os.environ.get('GALILEO_CONSOLE_URL', 'https://app.galileo.ai/').removesuffix('/')}/datasets/{dataset.id}")

### Expand the dataset with synthetic data

Galileo can also generate synthetic data to augment a dataset. This uses whatever LLM integration you have configured, along with a prompt and some examples to generate synthetic data for you. In an ideal world, you would always use production data that is based on real world actions from users, but in the absence of this, you can use synthetic data.

Synthetic data generation uses the model passed in to the `extend_dataset` call, which you set earlier.

Make sure to review the generated data once complete, to ensure you are happy with it.

In [ ]:
from galileo.datasets import extend_dataset

# Get the original dataset as a simple array to use as examples
existing_rows = [v.values[0] for v in dataset.get_content().rows]

# Generate synthetic data
new_rows = extend_dataset(
    prompt_settings={'model_alias': MODEL},
    prompt="Recipe chatbot",
    instructions="""
    Write queries to test various aspects of a recipe chatbot. Consider including requests related to:
    - Specific cuisines (e.g., Italian pasta dish, Spicy Thai curry)
    - Dietary restrictions (e.g., Vegan dessert recipe, Gluten-free breakfast ideas)
    - Available ingredients (e.g., What can I make with chicken, rice, and broccoli?)
    - Meal types (e.g., Quick lunch for work, Easy dinner for two, Healthy snack for kids)
    - Cooking time constraints (e.g., Recipe under 30 minutes)
    - Skill levels (e.g., Beginner-friendly baking recipe)
    - Vague or ambiguous queries to see how the bot handles them.
    """,
    examples=existing_rows,
    data_types=['General Query'],
    count=10,
)

# Add the new synthetic rows to the dataset
dataset = dataset.add_rows([{"input": row.values[0]} for row in new_rows])

print(f"Dataset extended with synthetic data. You can view it at {os.environ.get('GALILEO_CONSOLE_URL', 'https://app.galileo.ai/').removesuffix('/')}/datasets/{dataset.id}")

### Save the results as a CSV file

The original source data came from a CSV file. We've now extended this to include more rows, so we need to write this back to a CSV file should you want to use it with the original recipe bot app.

The original source CSV file was loaded from GitHub. Here we'll just write to the local file system and you can copy the generated file over the CSV file wherever you have cloned the recipe chatbot code.

In [ ]:
existing_rows = [v.values[0] for v in dataset.get_content().rows]

with open("sample_queries.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "query"])
    for i, val in enumerate(existing_rows, start=1):
        writer.writerow([i, val])

## Part 3: Run the Bulk Test & Evaluate

### Run the bulk test script using an experiment

The final step asks you to run the bulk test script. This script takes the dataset, uses the system prompt you defined earlier, then runs each row from the sample queries against that prompt, outputting the results.

We can do this in a more consistent fashion by running our experiment again with the new dataset. That way the results persist, and we can [compare the experiment runs](https://v2docs.galileo.ai/concepts/experiments/compare) to see the impact of system prompt changes.

Once the experiment is started, you will see a link in the output to monitor the progress and see the results.

In [ ]:
# Run the prompt experiment again with the updated prompt
run_experiment_using_prompt_and_dataset()

## Additional: Run an experiment against the original recipe bot

This code sample has a simulation of the recipe bot, using the system prompt and dataset to generate results. This works in this instance as the recipe bot is a simple call to an LLM using the prompt and query from the dataset.

In a real world scenario, you would probably want to run your actual application in the experiment, allowing you to evaluate everything that your application has, such as agents, tool calls, and more. Once you have this configured, this becomes something you can run as a unit test, or as part of a CI/CD pipeline. You can then also add evals to this, adding metrics to your experiment to evaluate each response.

To do this, you can pass a function to the `run_experiment` call. The steps to do this are:

- Define your dataset in advance, either in code or manually in the Galileo console. This becomes more important if you are using this in a CI/CD pipeline, where you can centrally manage a dataset of test cases
- Add a unit test to the recipe chatbot. This unit test uses the `run_experiment` call, but instead of passing in a prompt, it passes in a function. This function is the call to the recipe agent, calling the [`get_agent_response` function](https://github.com/ai-evals-course/recipe-chatbot/blob/35618065f209dbf27075b4a4183a986a0c10bd14/backend/utils.py#L31). You would probably need a wrapper function to ensure the dataset row and system prompt are passed in to this function correctly.
- Add Galileo logging where applicable inside the app to gather as much detail for the experiment trace as possible.

You can read more about running functions as experiments in the [Galileo experiments documentation](https://v2docs.galileo.ai/sdk-api/experiments/running-experiments#run-experiments-against-complex-code-with-custom-functions).